In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import zarr
import numcodecs
import gcsfs
import dask
import dask.array

In [2]:
JAMES_ZARR = 'solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/OSGB36/all_zarr'
OUT_ZARR = 'solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/OSGB36/all_zarr_int16'

gcs = gcsfs.GCSFileSystem()

## Read James' Zarr store

In [3]:
james_store = gcsfs.GCSMap(root=JAMES_ZARR, gcs=gcs)

In [4]:
%%time
# zarr.consolidate_metadata(james_store)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [5]:
%%time
james_xarray = xr.open_zarr(james_store, consolidated=True)

CPU times: user 4.26 s, sys: 422 ms, total: 4.68 s
Wall time: 2min 2s


In [6]:
# The first 58 images are all NaN.
james_xarray_da = james_xarray['stacked_eumetsat_data'][58:]

In [7]:
%%time
# nans_delayed = xr.ufuncs.isnan(james_xarray_da.any(dim=['variable', 'x', 'y']))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [8]:
%%time
# james_xarray_da = james_xarray_da[~nans_delayed]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [9]:
%%time
# nans = xr.ufuncs.isnan(james_xarray_da[:10000, :, :, 0]).any(dim=['x', 'y']).compute()
# nans

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [10]:
# nans.time[nans]

In [11]:
%%time
# dropped = james_xarray_da.dropna(dim='time')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [12]:
minimums = james_xarray_da.min(axis=[0, 1, 2])
maximums = james_xarray_da.max(axis=[0, 1, 2])

In [13]:
%%time
minimums, maximums = dask.compute(minimums, maximums)

/home/JackKelly/miniconda3/envs/storage_benchmarks/lib/python3.8/site-packages/dask/utils.py:29: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/JackKelly/miniconda3/envs/storage_benchmarks/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: All-NaN slice encountered
  return func(*(_execute_task(a, cache) for a in args))
_call non-retriable exception: 
Traceback (most recent call last):
  File "/home/JackKelly/miniconda3/envs/storage_benchmarks/lib/python3.8/site-packages/gcsfs/core.py", line 507, in _call
    self.validate_response(status, contents, json, path, headers)
  File "/home/JackKelly/miniconda3/envs/storage_benchmarks/lib/python3.8/site-packages/gcsfs/core.py", line 1230, in validate_response
    raise HttpError({"code": status})
gcsfs.utils.HttpError


HttpError: 

In [14]:
minimums

<xarray.DataArray 'stacked_eumetsat_data' (variable: 12)>
dask.array<nanmin-aggregate, shape=(12,), dtype=float32, chunksize=(12,), chunktype=numpy.ndarray>
Coordinates:
  * variable  (variable) <U6 'HRV' 'IR_016' 'IR_039' ... 'WV_062' 'WV_073'

In [15]:
maximums

<xarray.DataArray 'stacked_eumetsat_data' (variable: 12)>
dask.array<nanmax-aggregate, shape=(12,), dtype=float32, chunksize=(12,), chunktype=numpy.ndarray>
Coordinates:
  * variable  (variable) <U6 'HRV' 'IR_016' 'IR_039' ... 'WV_062' 'WV_073'

In [16]:
%%time
james_xarray_da -= minimums

CPU times: user 36.8 ms, sys: 56 µs, total: 36.9 ms
Wall time: 35.2 ms


In [17]:
new_max = maximums - minimums

In [18]:
%%time
james_xarray_da /= new_max

CPU times: user 34.9 ms, sys: 130 µs, total: 35 ms
Wall time: 33.5 ms


In [19]:
# james_xarray_da = james_xarray_da.clip(min=0, max=1)

In [20]:
BITS_PER_PIXEL = 10
upper_bound = (2 ** BITS_PER_PIXEL) - 1

james_xarray_da *= upper_bound

In [21]:
%%time
james_xarray_da = james_xarray_da.fillna(-1)

CPU times: user 92.8 ms, sys: 248 µs, total: 93 ms
Wall time: 90.4 ms


In [22]:
james_xarray_da = james_xarray_da.round().astype(np.int16)

In [23]:
_, y_size, x_size, _ = james_xarray_da.shape

In [24]:
out_store = gcsfs.GCSMap(root=OUT_ZARR, gcs=gcs)

In [25]:
%%time

chunks = (36, y_size, x_size, 1)

ds = xr.Dataset(
    {'stacked_eumetsat_data': james_xarray_da[:10].chunk(chunks)})

encoding = {'stacked_eumetsat_data': {
    'compressor': numcodecs.Blosc(cname="zstd", clevel=5),
    'chunks': chunks
}}
ds.to_zarr(out_store, mode='w', encoding=encoding)

KeyError: 

# Read

In [ ]:
read_ds = xr.open_zarr(out_store)

In [ ]:
%%time
data = read_ds['stacked_eumetsat_data'][:100, :, :, 0].compute()

## OLD

In [ ]:
def open_dask_array(store, dataset='HRV'):
    zarr_store = zarr.open_consolidated(store)
    zarr_array = zarr_store[dataset]
    print(zarr_array.chunks)
    return dask.array.from_zarr(zarr_array)

In [ ]:
%%time
james_dask = open_dask_array(james_store, dataset='stacked_eumetsat_data')

In [ ]:
james_dask

In [ ]:
minimums = dask.array.nanmin(james_dask[:10000:90, :, :, :], axis=[0, 1, 2]).compute()
minimums

In [ ]:
maximums = dask.array.nanmax(james_dask[:10000:90, :, :, :], axis=[0, 1, 2]).compute()
maximums

In [ ]:
new_maxes = maximums - minimums
new_maxes

In [ ]:
james_dask_normalised = (james_dask - minimums) / new_maxes

In [ ]:
dask.array.nanmin(james_dask_normalised[:10000:90, :, :, :], axis=[0, 1, 2]).compute()

In [ ]:
dask.array.nanmax(james_dask_normalised[:10000:90, :, :, :], axis=[0, 1, 2]).compute()

In [ ]:
BITS_PER_PIXEL = 10
upper_bound = (2 ** BITS_PER_PIXEL) - 1

In [ ]:
james_dask_normalised[dask.array.isnan(james_dask_normalised)][:180:90, :, :, :].compute()

In [ ]:
dask.array.nan_to_num(james_dask_normalised[:180:90, :, :, :], -1).compute()

In [ ]:
james_dask_stretched = (james_dask_normalised * upper_bound).round().fillna(-1).astype(np.int16)

In [ ]:
james_dask_stretched[:28, :, :, 0].compute()

In [ ]:
dask.array.nanmin(james_dask_stretched[:10000:90, :, :, :], axis=[0, 1, 2]).compute()

In [ ]:
dask.array.nanmax(james_dask_stretched[:10000:90, :, :, :], axis=[0, 1, 2]).compute()

## Save to GCP as Zarr

In [ ]:
def change_bits_per_pixel(da: xr.DataArray, bits_per_pixel: int) -> xr.DataArray:
    da = da.copy(deep=True)
    da -= da.min()
    da /= da.max()
    upper_bound = (2 ** bits_per_pixel) - 1
    da *= upper_bound
    da = da.round()
    assert da.min() == 0
    assert da.max() == upper_bound
    return da


def save_to_zarr(da: xr.DataArray, store, compressor=None, filters=None, chunks=None):
    DA_NAME = 'HRV'
    ds = xr.Dataset({DA_NAME: da})
    encoding = {DA_NAME: {'compressor': compressor, 'filters': filters}}
    if chunks is not None:
        encoding[DA_NAME]['chunks'] = chunks
    ds.to_zarr(store, mode='w', encoding=encoding)

In [ ]:
concat_da.shape

In [ ]:
store = gcsfs.GCSMap(root=ZARR_PATH, gcs=gcs)

In [ ]:
%%time
def save():
    da = change_bits_per_pixel(concat_da, bits_per_pixel=12)
    da = da.astype(np.uint16)
    save_to_zarr(
        da, store, 
        compressor=numcodecs.Blosc(cname="zstd", clevel=5),
        chunks=(128, 256, 256)  # default = (27, 163, 163)
    )

save()

In [ ]:
%%time
zarr.consolidate_metadata(store)

In [ ]:
%%time
du_mb_total = gcs.du(ZARR_PATH) / 1E6

In [ ]:
print(f'{du_mb_total:,.1f} MBytes')

In [ ]:
du_mb_per_chunk = gcs.du(ZARR_PATH + '/HRV/0.0.0') / 1E6
print(f'{du_mb_per_chunk:,.1f} MB per chunk')

## Read from GCP (single process; single thread)

In [ ]:
def load_all(ds):
    ds.load()
    

def load_rectangle(ds):
    ds_rect = read_ds['HRV'][:, 128:256, 128:256]
    ds_rect.load()

In [ ]:
NUM_RUNS = 3
col_to_func = {'all': load_all, 'rectangle': load_rectangle}
runtimes = pd.DataFrame(np.NaN, columns=col_to_func.keys(), index=range(NUM_RUNS))

for run_i in range(NUM_RUNS):
    print(run_i)
    for col, func in col_to_func.items():
        print(col, end=': ')
        read_ds = xr.open_zarr(store, consolidated=True)
        t0 = time.time()
        func(read_ds)
        t1 = time.time()
        runtime = t1 - t0
        print(f'{runtime:.1f} seconds')
        runtimes[col][run_i] = runtime

In [ ]:
runtimes

In [ ]:
runtimes.mean()

In [ ]:
# _ = zarr.blosc.set_nthreads(4)
zarr.blosc.get_nthreads()

## Read from GCP (using dask)

In [ ]:
def open_dask_array(store):
    zarr_store = zarr.open_consolidated(store)
    zarr_array = zarr_store['HRV']
    print(zarr_array.chunks)
    return dask.array.from_zarr(zarr_array)

In [ ]:
%%time
dask_zarr = open_dask_array(store)

In [ ]:
dask_zarr

In [ ]:
dask_slice = dask_zarr[:, 128:256, 128:256]

In [ ]:
# dask_slice.visualize(filename='dask_graph.svg')

In [ ]:
%%time
dask_slice_materialised = dask_slice.compute() # scheduler='single-threaded') # (scheduler='threads')